# Datajoint_sandbox


In [1]:
import pandas as pd
import numpy as np
import os
import datajoint as dj
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
matplotlib.style.use('ggplot')
%matplotlib inline

<a id='connect'></a>
# Connect to the datajoint00 database
Username and password are same as Princeton login.

**Note**: You need to be on campus or using a VPN to connect to the database.

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [3]:
db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
# db_spockadmin = dj.create_virtual_module('ahoag_spockadmin_demo','ahoag_spockadmin_demo')
# db_lightsheet = dj.create_virtual_module('a','u19lightserv_lightsheet')

In [4]:
restrict_dict = {'username':'lightserv-test','request_name':'test_new_imaging_request'}
db_lightsheet.Request() & restrict_dict

username user in the lab,request_name,requested_by user in the lab,date_submitted The date it was submitted as a request,time_submitted The time it was submitted as a request,labname,correspondence_email,description,species,number_of_samples,testing,is_archival,raw_data_retention_preference
lightserv-test,test_new_imaging_request,lightserv-test,2021-01-06,15:06:01,braincogs,lightserv-test@princeton.edu,test,mouse,2,0,0,important


In [5]:
db_lightsheet.Request.ImagingRequest & restrict_dict

username user in the lab,request_name,sample_name,imaging_request_number,imager user in the lab,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress
lightserv-test,test_new_imaging_request,test_new_imaging_request-001,1,None,2021-01-06,15:06:01,None,incomplete
lightserv-test,test_new_imaging_request,test_new_imaging_request-001,2,None,2021-01-06,15:14:49,None,incomplete
lightserv-test,test_new_imaging_request,test_new_imaging_request-002,1,None,2021-01-06,15:06:01,None,incomplete
lightserv-test,test_new_imaging_request,test_new_imaging_request-002,2,None,2021-01-06,15:14:49,None,incomplete


In [6]:
db_lightsheet.Request.ImagingBatch() & restrict_dict

username user in the lab,request_name,imaging_batch_number,imaging_request_number,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_request_date_submitted date that the user submitted the request for imaging,imaging_request_time_submitted time that the user submitted the request for imaging,imaging_performed_date date that the imaging form was submitted by the imager,imaging_progress,imaging_dict
lightserv-test,test_new_imaging_request,1,1,ahoag,1,2021-01-06,15:06:01,None,in progress,=BLOB=
lightserv-test,test_new_imaging_request,1,2,None,1,2021-01-06,15:14:49,None,incomplete,=BLOB=
lightserv-test,test_new_imaging_request,2,1,None,1,2021-01-06,15:06:01,None,incomplete,=BLOB=
lightserv-test,test_new_imaging_request,2,2,None,1,2021-01-06,15:14:49,None,incomplete,=BLOB=


In [7]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()

imaging_request_contents = (clearing_batch_contents * sample_contents * \
    request_contents * imaging_batch_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species','number_in_imaging_batch',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)')

In [8]:
imaging_request_contents = dj.U('username','request_name',
                                'imaging_batch_number',
                                'imaging_request_number').aggr(
    imaging_request_contents,clearer='clearer',
    clearing_progress='clearing_progress',
    datetime_submitted='datetime_submitted',
    imaging_progress='imaging_progress',imager='imager',
    species='species',number_in_imaging_batch='number_in_imaging_batch',
    all_samples_cleared='SUM(IF(clearing_progress="complete",1,0))=count(*)')

In [10]:
imaging_request_contents & restrict_dict

username user in the lab,request_name,imaging_batch_number,imaging_request_number,clearing_progress,clearer user in the lab,species,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_progress,datetime_submitted calculated attribute,all_samples_cleared calculated attribute
lightserv-test,test_new_imaging_request,1,1,complete,ahoag,mouse,ahoag,1,in progress,2021-01-06 15:06:01,1
lightserv-test,test_new_imaging_request,1,2,complete,ahoag,mouse,None,1,incomplete,2021-01-06 15:14:49,1
lightserv-test,test_new_imaging_request,2,1,complete,ahoag,mouse,None,1,incomplete,2021-01-06 15:06:01,1
lightserv-test,test_new_imaging_request,2,2,complete,ahoag,mouse,None,1,incomplete,2021-01-06 15:14:49,1


In [17]:
imaging_request_contents & restrict_dict

username user in the lab,request_name,imaging_batch_number,imaging_request_number,clearing_progress,clearer user in the lab,species,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_progress,datetime_submitted calculated attribute,all_samples_cleared calculated attribute
lightserv-test,test,1,1,incomplete,lightserv-test,mouse,lightserv-test,1,incomplete,2021-01-05 15:31:35,0
lightserv-test,test,1,2,incomplete,lightserv-test,mouse,None,1,incomplete,2021-01-06 12:15:14,0
lightserv-test,test,2,1,incomplete,lightserv-test,mouse,lightserv-test,1,incomplete,2021-01-05 15:31:35,0


In [11]:
contents_being_imaged = imaging_request_contents & 'all_samples_cleared=1' & \
    'imaging_progress="in progress"'

In [12]:
contents_being_imaged & restrict_dict

username user in the lab,request_name,imaging_batch_number,imaging_request_number,clearing_progress,clearer user in the lab,species,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_progress,datetime_submitted calculated attribute,all_samples_cleared calculated attribute
lightserv-test,test_new_imaging_request,1,1,complete,ahoag,mouse,ahoag,1,in progress,2021-01-06 15:06:01,1


In [22]:
imaging_request_contents & 'imaging_request_number=2'

username user in the lab,request_name,imaging_batch_number,imaging_request_number,clearing_progress,clearer user in the lab,species,imager user in the lab,number_in_imaging_batch date that the imaging form was submitted by the imager,imaging_progress,datetime_submitted calculated attribute,all_samples_cleared calculated attribute
lightserv-test,test,1,2,complete,lightserv-test,mouse,None,1,incomplete,2021-01-06 12:15:14,1


In [23]:
sample_restrict_dict = \
    {'sample_name': 'test-001', 'imaging_request_number': '2', 'imaging_batch_number': '1'}

In [25]:
db_lightsheet.Request.ClearingBatchSample()

username user in the lab,request_name,sample_name,clearing_protocol,antibody1,antibody2,clearing_batch_number


In [5]:
(db_lightsheet.Request() & 'username="pbibawi"').fetch('request_name')

array(['20190405_pb_X015,_20190405_pb_X045,_20190405_pb_A233,20190405_pb',
       '20191001_pb_Z265_ctb647_rPPC,_20191001_pb_Z266_ctb647_lPPC,_2019',
       'pb_udisco_647', 'pb_udisco_647_488_4x'], dtype=object)

In [4]:
username='willmore'
request_name='201106_willmore'
sample_name = '201106_willmore-303'
imaging_request_number=1
processing_request_number=1
processing_restrict_dict = dict(username=username,request_name=request_name,
                sample_name=sample_name,imaging_request_number=imaging_request_number,
                processing_request_number=processing_request_number)

processing_request_contents = db_lightsheet.Request.ProcessingRequest() & processing_restrict_dict

processing_resolution_request_contents = db_lightsheet.Request.ProcessingResolutionRequest() & \
    processing_restrict_dict

processing_channel_contents = db_lightsheet.Request.ProcessingChannel() & processing_restrict_dict
image_resolution='1.1x'
processing_channel_contents_this_resolution = (processing_channel_contents & \
            f'image_resolution="{image_resolution}"')
restrict_dict = {'channel_name':'488','ventral_up':0}
this_processing_channel_content = (processing_channel_contents_this_resolution & \
                    restrict_dict)
this_processing_channel_content

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,processing_request_number,lightsheet_channel_str,imspector_version,datetime_processing_started,datetime_processing_completed,intensity_correction,metadata_xml_string The entire metadata xml string. Sometimes it is not available so those times it will be NULL,left_lightsheet_stitched_precomputed_spock_jobid,left_lightsheet_stitched_precomputed_spock_job_progress,right_lightsheet_stitched_precomputed_spock_jobid,right_lightsheet_stitched_precomputed_spock_job_progress,blended_precomputed_spock_jobid,blended_precomputed_spock_job_progress,downsized_precomputed_spock_jobid,downsized_precomputed_spock_job_progress,registered_precomputed_spock_jobid,registered_precomputed_spock_job_progress
willmore,201106_willmore,201106_willmore-303,1,1.1x,488,0,1,regch,,2020-12-06 12:16:01,None,1,=BLOB=,None,None,None,None,20558099,COMPLETED,20558199,COMPLETED,20558197,COMPLETED


In [53]:
username='willmore'
request_name='201106_willmore'
imaging_batch_number=1
imaging_request_number=1

restrict_dict = dict(username=username,request_name=request_name,
                imaging_batch_number=imaging_batch_number)

channel_contents_all_samples = (db_lightsheet.Request.ImagingChannel() & f'request_name="{request_name}"' & \
    f'username="{username}"' )
imaging_request_contents = db_lightsheet.Request.ImagingRequest() & \
    restrict_dict
imaging_resolution_contents = db_lightsheet.Request.ImagingResolutionRequest() & \
    restrict_dict
imaging_resolution_contents

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,microscope,notes_for_imager,notes_from_imaging
willmore,201106_willmore,201106_willmore-301,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-302,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-303,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-312,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-313,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-314,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-425,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",
willmore,201106_willmore,201106_willmore-426,1,1.1x,LaVision,"Image both with dorsal and ventral sides down. We have had issues with shadows obscuring the fiber ends. Also to make nicer images, please also use red/yellow laser",


In [44]:
channel_contents_all_samples

username user in the lab,request_name,sample_name,imaging_request_number,image_resolution,channel_name,ventral_up whether brain was flipped upside down to be imaged,imaging_date,zoom_body_magnification only applicable for 2x,left_lightsheet_used,right_lightsheet_used,registration,injection_detection,probe_detection,cell_detection,generic_imaging,pixel_type,image_orientation how the imager imaged the sample. Most of the time will be horizontal,numerical_aperture it is not always recorded in metadata so those times it will be NULL,tiling_scheme,tiling_overlap,z_step distance between z planes in microns,number_of_z_planes,rawdata_subfolder,"imspector_channel_index refers to multi-channel imaging - 0 if first (or only) channel in rawdata_subfolder, 1 if second, 2 if third, ...",left_lightsheet_precomputed_spock_jobid,left_lightsheet_precomputed_spock_job_progress,right_lightsheet_precomputed_spock_jobid,right_lightsheet_precomputed_spock_job_progress
willmore,201106_willmore,201106_willmore-301,1,1.1x,488,0,None,1.0,1,1,1,0,1,0,0,None,horizontal,nan,1x1,0.0,10.0,836,201123_willmore-301_1_1x_488_016na_1hfds_z10um_50msec_17-43-53,0,20416000,COMPLETED,20415998,COMPLETED
willmore,201106_willmore,201106_willmore-301,1,1.1x,488,1,None,1.0,1,1,1,0,1,0,0,None,horizontal,nan,1x1,0.0,10.0,725,201124_willmore-301_1_1x_488_016na_1hfds_z10um_50msec_ventralup_11-08-53,0,20416003,COMPLETED,20415999,COMPLETED
willmore,201106_willmore,201106_willmore-301,1,1.1x,555,0,None,1.0,1,1,0,0,1,0,1,None,horizontal,nan,1x1,0.0,10.0,836,201123_willmore-301_1_1x_555_016na_1hfds_z10um_50msec_17-57-17,0,20416012,COMPLETED,20416011,COMPLETED
willmore,201106_willmore,201106_willmore-301,1,1.1x,555,1,None,1.0,1,1,0,0,1,0,1,None,horizontal,nan,1x1,0.0,10.0,725,201124_willmore-301_1_1x_555_016na_1hfds_z10um_50msec_ventralup_11-15-40,0,20416010,COMPLETED,20416015,COMPLETED
willmore,201106_willmore,201106_willmore-302,1,1.1x,488,0,None,1.0,1,1,1,0,1,0,0,None,horizontal,nan,1x1,0.0,10.0,604,201124_willmore-302_1_1x_488_016na_1hfds_z10um_50msec_12-45-54,0,20416024,COMPLETED,20416023,COMPLETED
willmore,201106_willmore,201106_willmore-302,1,1.1x,488,1,None,1.0,1,1,1,0,1,0,0,None,horizontal,nan,1x1,0.0,10.0,616,201124_willmore-302_1_1x_488_016na_1hfds_z10um_50msec_ventralup_13-00-40,0,20416020,COMPLETED,20416027,COMPLETED
willmore,201106_willmore,201106_willmore-302,1,1.1x,555,0,None,1.0,1,1,0,0,1,0,1,None,horizontal,nan,1x1,0.0,10.0,604,201124_willmore-302_1_1x_555_016na_1hfds_z10um_50msec_12-52-13,0,20416034,COMPLETED,20416036,COMPLETED
willmore,201106_willmore,201106_willmore-302,1,1.1x,555,1,None,1.0,1,1,0,0,1,0,1,None,horizontal,nan,1x1,0.0,10.0,616,201124_willmore-302_1_1x_555_016na_1hfds_z10um_50msec_ventralup_13-06-23,0,20416035,COMPLETED,20416039,COMPLETED
willmore,201106_willmore,201106_willmore-303,1,1.1x,488,0,None,1.0,1,1,1,0,1,0,0,None,horizontal,nan,1x1,0.0,10.0,702,201124_willmore-303_1_1x_488_016na_1hfds_z10um_50msec_12-00-22,0,20416043,COMPLETED,20416048,COMPLETED
willmore,201106_willmore,201106_willmore-303,1,1.1x,488,1,None,1.0,1,1,1,0,1,0,0,None,horizontal,nan,1x1,0.0,10.0,652,201124_willmore-303_1_1x_488_016na_1hfds_z10um_50msec_ventralup_12-17-03,0,20416047,COMPLETED,20416051,COMPLETED


In [45]:
image_resolutions_no_processing = ["2x","3.6x","15x"]

In [46]:
batch_unique_image_resolutions = set(imaging_resolution_contents.fetch('image_resolution'))
# batch_unique_image_re?solutions = set(['1.1x','3.6x'])

In [47]:
have_processing_requests = any([x not in image_resolutions_no_processing for x in batch_unique_image_resolutions])
have_processing_requests

True

In [54]:
set(imaging_request_contents.fetch('sample_name'))


{'201106_willmore-301',
 '201106_willmore-302',
 '201106_willmore-303',
 '201106_willmore-312',
 '201106_willmore-313',
 '201106_willmore-314',
 '201106_willmore-425',
 '201106_willmore-426'}

In [48]:
sample_names = set(imaging_resolution_contents.fetch('sample_name'))
sample_names

{'201106_willmore-301',
 '201106_willmore-302',
 '201106_willmore-303',
 '201106_willmore-312',
 '201106_willmore-313',
 '201106_willmore-314',
 '201106_willmore-425',
 '201106_willmore-426'}

In [49]:
data_rootpath = '/jukebox/LightSheetData/lightserv'

In [52]:
message_body = ('Hello!\n\nThis is an automated email sent from lightserv, '
                        'the Light Sheet Microscopy portal at the Histology and Brain Registration Core Facility. '
                        'The raw data your request:\n'
                        f'request_name: "my-request"\n'
                        f'are now available on bucket here: \n\n')
for sample_name in sample_names:
    sample_basepath = os.path.join(data_rootpath,username,request_name,
    sample_name,f'imaging_request_number_{imaging_request_number}',  
    'rawdata')
    channel_contents_this_sample = channel_contents_all_samples & f'sample_name="{sample_name}"' 
    message_body += f'Sample name: {sample_name}:\n'
    for channel_dict in channel_contents_this_sample:
        channel_name = channel_dict['channel_name']
        image_resolution = channel_dict['image_resolution']
        ventral_up = channel_dict['ventral_up']
        rawdata_subfolder = channel_dict['rawdata_subfolder']
        if ventral_up:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}_ventral_up',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name} (ventral up): {rawdata_fullpath}\n'
        else:
            rawdata_fullpath = os.path.join(
                data_rootpath,username,request_name,sample_name,
                f'imaging_request_{imaging_request_number}',
                'rawdata',f'resolution_{image_resolution}',
                rawdata_subfolder)
            message_body += f'\tchannel {channel_name}: {rawdata_fullpath}\n'
    message_body += '\n'
message_body += '\n'
if have_processing_requests:
    message_body += ('To start processing your data, '
            f'go to the processing management GUI: /some/path '
            'and find your sample to process.\n\n')
message_body += 'Thanks,\n\nThe Core Facility'
print(message_body)

Hello!

This is an automated email sent from lightserv, the Light Sheet Microscopy portal at the Histology and Brain Registration Core Facility. The raw data your request:
request_name: "my-request"
are now available on bucket here: 

Sample name: 201106_willmore-426:
	channel 488: /jukebox/LightSheetData/lightserv/willmore/201106_willmore/201106_willmore-426/imaging_request_1/rawdata/resolution_1.1x/201124_willmore-426_1_1x_488_016na_1hfds_z10um_50msec_15-37-10
	channel 488 (ventral up): /jukebox/LightSheetData/lightserv/willmore/201106_willmore/201106_willmore-426/imaging_request_1/rawdata/resolution_1.1x_ventral_up/201124_willmore-426_1_1x_488_016na_1hfds_z10um_50msec_ventral_15-12-26
	channel 555: /jukebox/LightSheetData/lightserv/willmore/201106_willmore/201106_willmore-426/imaging_request_1/rawdata/resolution_1.1x/201124_willmore-426_1_1x_555_016na_1hfds_z10um_50msec_15-42-24
	channel 555 (ventral up): /jukebox/LightSheetData/lightserv/willmore/201106_willmore/201106_willmore-426

In [ ]:
this_processing_channel_content_dict = this_processing_channel_content.fetch1()
this_processing_channel_content_dict

In [ ]:
processing_channel_contents_this_resolution

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest().fetch('atlas_name',limit=1)[0]

In [ ]:
processing_resolution_request_contents

In [ ]:
request_contents = db_lightsheet.Request()
sample_contents = db_lightsheet.Request.Sample()
clearing_batch_contents = db_lightsheet.Request.ClearingBatch()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
imaging_channel_contents = db_lightsheet.Request.ImagingChannel()

In [ ]:
imaging_channel_contents & 'ventral_up=1'

In [ ]:
processing_resolution_contents = db_lightsheet.Request.ProcessingResolutionRequest() & {'username':'willmore'}
processing_resolution_contents 

In [ ]:
dj.Table._update(processing_resolution_contents,'atlas_name','allen_2017')

In [ ]:
clearing_batch_contents

In [ ]:
combined_contents = (clearing_batch_contents * request_contents).proj(
'description','number_in_batch','expected_handoff_date',
'clearing_protocol','species',
'clearer','clearing_progress','clearing_protocol','antibody1','antibody2',
datetime_submitted='TIMESTAMP(date_submitted,time_submitted)')
''' First get all entities that are currently being cleared '''

In [ ]:
idiscoplus_cleared_contents = combined_contents & 'clearing_protocol="iDISCO+_immuno"' & 'clearing_progress="complete"'
idiscoplus_cleared_contents

In [ ]:
df_idiscoplus_cleared = (idiscoplus_cleared_contents).fetch(
format='frame',order_by='datetime_submitted DESC')

In [ ]:
df_idiscoplus_cleared.to_csv("idiscoplus_already_cleared_asof_2020Nov12.csv")

In [ ]:
idisco_contents = db_lightsheet.Request.IdiscoPlusClearing()
idisco_contents

In [ ]:
idisco_website_cleared_contents = idisco_contents * idiscoplus_cleared_contents
idisco_website_cleared_contents

In [ ]:
idisco_results=idisco_website_cleared_contents.fetch(as_dict=True)
idisco_results

In [ ]:
df_idisco=pd.DataFrame(idisco_results)
df_idisco

In [ ]:
df_idisco.to_csv('idiscoplus_antibody_notes_already_cleared_asof_2020Nov12.csv')

In [ ]:
imaging_channel_contents & \
    {'username': 'ahoag',
     'request_name': 'test123',
     'sample_name': 'test123-001',
     'imaging_request_number': 1,
     'image_resolution': '1.3x',
     'channel_name': '488'}

In [ ]:
clearing_batch_contents.fetch(as_dict=True,limit=2)

In [ ]:
clearing_batch_contents.fetch?

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples4"' 

In [ ]:
insert_dict = {'username': 'ahoag', 'request_name': 'test_many_samples4',
               'imaging_request_number': 1, 'image_resolution': '1.1x',
               'channel_name': '555',
               'injection_detection': 1,
               'sample_name': 'test_many_samples4-001'}


In [ ]:
imaging_merged_contents = (imaging_request_contents \
    * clearing_batch_contents * sample_contents * request_contents * imaging_channel_contents).\
    proj('clearer','clearing_progress',
    'imaging_request_date_submitted','imaging_request_time_submitted',
    'imaging_progress','imager','species',
    'image_resolution','channel_name',
    datetime_submitted='TIMESTAMP(imaging_request_date_submitted,imaging_request_time_submitted)') & \
    {'username':'ahoag',}

In [ ]:
replicated_args = dict(username='username',species='species',
                       datetime_submitted='datetime_submitted',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
    imaging_merged_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',**replicated_args,
    image_resolution='image_resolution').\
    proj(comb_col='CONCAT(image_resolution,",",new_col)',**replicated_args,)
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    **replicated_args,
    final_col='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
dj.U('request_name','imaging_request_number','final_col').aggr(interm2,                               
    **replicated_args,
    number_in_batch='COUNT(*)')

In [ ]:
replicated_args = dict(username='username',
                       imager='imager',imaging_progress='imaging_progress')

In [ ]:
interm=dj.U("username","request_name","sample_name","imaging_request_number").aggr(
                        imaging_channel_contents,new_col='GROUP_CONCAT(channel_name ORDER BY channel_name)',
                        image_resolution='image_resolution').\
                        proj(comb_col='CONCAT(image_resolution,",",new_col)')
interm

In [ ]:
interm2=dj.U('request_name','sample_name','imaging_request_number').aggr(interm,
    sample_name='MIN(sample_name)',imaging_str='GROUP_CONCAT(comb_col)')
interm2

In [ ]:
interm3=dj.U('request_name','imaging_request_number','imaging_str').aggr(interm2,                               
    number_in_batch='COUNT(*)',).proj('request_name','number_in_batch')
interm3

In [ ]:
request_counter_dict = {}

In [ ]:
interm3.fetch(as_dict=True)

In [ ]:
imaging_channel_contents & 'request_name="test_many_samples3"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents = db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents.fetch?

In [ ]:
db_lightsheet.Request.ImagingChannel().insert(channel_entry_dicts,skip_duplicates=True)

In [ ]:
db_lightsheet.Request.ImagingChannel() & 'request_name="test_many_samples2"'

In [ ]:
sorted(set(["a","b","a","a"]))

In [ ]:
db_lightsheet.Request.ProcessingResolutionRequest() & 'request_name="test_many_samples2"'

In [ ]:
db_lightsheet.Request.ProcessingChannel() & 'request_name="test_many_samples2"'

In [ ]:
contents 

In [ ]:
var=4
s = f'{{var}} is the number'
s

In [ ]:
b = '1.3x_ventral_up'

In [ ]:
b if 'ventral_up' not in b else b.strip("_ventral_up")

In [ ]:
b.strip("_ventral_up")

In [ ]:
l=[{'id':'A','image_resolution':'4x','ventral_up':True},
   {'id':'B','image_resolution':'4x','ventral_up':False},
    {'id':'C','image_resolution':'1.3x','ventral_up':False},
    {'id':'D','image_resolution':'1.3x','ventral_up':False},
    {'id':'E','image_resolution':'1.3x','ventral_up':True}
  ]
tup_list = [(d['image_resolution'],d['ventral_up']) for d in l]
sorted(set(tup_list))

In [ ]:
all_channel_contents = imaging_channel_contents & 'request_name="test_dorsal_ventral"'

In [ ]:
results_list=all_channel_contents.fetch('image_resolution','ventral_up',as_dict=True)
tup_list = [(d['image_resolution'],d['ventral_up']) for d in results_list]
set(tup_list)
# all_channel_contents.fetch

In [ ]:
for a,b, in tup_list:
    print(a,b)

In [ ]:
bool(int('0'))

In [ ]:
job_contents = db_spockadmin.ProcessingPipelineSpockJob()
job_contents

In [ ]:
unique_contents = dj.U('jobid_step0','username',).aggr(
		job_contents,timestamp='max(timestamp)')*job_contents
unique_contents

In [ ]:
int(False)

In [ ]:
int(bool("False"))

In [ ]:
bool("False")